<a href="https://colab.research.google.com/github/lin941006/Time/blob/main/%E5%80%8B%E4%BA%BA%E6%99%82%E9%96%93%E8%A6%8F%E5%8A%83_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio pandas
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta

# =========================
# 資料暫存
# =========================
tasks = []

# =========================
# 行程類型與待辦事項
# =========================
category_choices = [
    "生活類",
    "工作/學習類",
    "通勤類",
    "休閒娛樂類",
    "社交互動類",
    "運動與健康類",
    "家務/生活管理類",
    "其他"
]

subcategory_choices = {
    "生活類": ["睡覺", "起床", "吃飯", "洗澡", "休息/發呆"],
    "工作/學習類": ["上班", "上課", "寫作業", "自習", "開會", "報告", "考試", "線上課程"],
    "通勤類": ["捷運", "公車", "火車", "走路", "騎車", "開車"],
    "休閒娛樂類": ["玩手機", "看電影", "追劇", "聽音樂", "看書", "逛街", "看展覽"],
    "社交互動類": ["聊天", "打電話", "聚會", "約會", "家庭時間", "同事交流"],
    "運動與健康類": ["健身", "跑步", "散步", "拉筋", "瑜珈", "冥想", "看醫生"],
    "家務/生活管理類": ["打掃", "洗衣服", "曬衣服", "煮飯", "採買食材", "整理房間", "倒垃圾"]
}

# =========================
# 時間設定
# =========================
time_choices = [f"{str(h).zfill(2)}:{m}" for h in range(24) for m in ["00", "30"]]
duration_choices = [i * 0.5 for i in range(1, 17)]  # 0.5 ~ 8 小時

# =========================
# 控制待辦事項顯示
# =========================
def toggle_subcategory(category, _):
    if category == "其他":
        return (
            gr.update(visible=False, value=None),
            gr.update(visible=True, value="")
        )
    elif category:
        return (
            gr.update(choices=subcategory_choices.get(category, []), visible=True, value=None),
            gr.update(visible=False, value="")
        )
    else:
        return (
            gr.update(visible=False, value=None),
            gr.update(visible=False, value="")
        )

# =========================
# 新增待辦事項
# =========================
def add_task(category, subcategory, other_input, hours):
    if not category or hours is None:
        return pd.DataFrame(tasks), None, None, "", None, gr.update(choices=[t["事項"] for t in tasks], value=None)

    task_name = other_input.strip() if category == "其他" else subcategory
    if not task_name:
        return pd.DataFrame(tasks), None, None, "", None, gr.update(choices=[t["事項"] for t in tasks], value=None)

    tasks.append({
        "事項": task_name,
        "所需時間(小時)": hours
    })

    return (
        pd.DataFrame(tasks),
        None,
        None,
        "",
        None,
        gr.update(choices=[t["事項"] for t in tasks], value=None)
    )

# =========================
# 產生今日行程
# =========================
def generate_schedule(start_time):
    if not tasks or not start_time:
        return "請先新增待辦事項並選擇開始時間。"

    current_time = datetime.strptime(start_time, "%H:%M")
    text = "📅 今日行程安排：\n\n"

    for task in tasks:
        end_time = current_time + timedelta(hours=task["所需時間(小時)"])
        text += f"{current_time.strftime('%H:%M')} ～ {end_time.strftime('%H:%M')}：{task['事項']}\n"
        current_time = end_time

    return text

# =========================
# 刪除指定行程（同步更新行程）
# =========================
def delete_task(selected_task, start_time):
    global tasks
    if selected_task:
        tasks = [t for t in tasks if t["事項"] != selected_task]

    schedule = generate_schedule(start_time) if start_time else ""
    return (
        pd.DataFrame(tasks),
        gr.update(choices=[t["事項"] for t in tasks], value=None),
        schedule
    )

# =========================
# 清除全部
# =========================
def clear_all():
    tasks.clear()
    return (
        pd.DataFrame(tasks),
        None,
        None,
        "",
        None,
        gr.update(choices=[], value=None),
        None,
        ""
    )

# =========================
# Gradio 介面
# =========================
with gr.Blocks(title="個人時間管理工具") as demo:
    gr.Markdown("## 🕒 個人時間管理工具")

    # 第一排
    with gr.Row():
        category_input = gr.Dropdown(category_choices, value=None, label="行程類型")
        subcategory_input = gr.Dropdown([], value=None, label="待辦事項", visible=False)
        other_input = gr.Textbox(label="其他事項", visible=False)
        time_input = gr.Dropdown(duration_choices, value=None, label="預計花費時間（小時）")

    category_input.change(
        toggle_subcategory,
        inputs=[category_input, subcategory_input],
        outputs=[subcategory_input, other_input]
    )

    # 第二排
    add_btn = gr.Button("➕ 新增今日待辦事項")

    # 第三排
    task_table = gr.Dataframe(label="📋 今日待辦清單", interactive=False)

    # 第四排
    start_time = gr.Dropdown(time_choices, value=None, label="開始時間")

    # 第五排
    schedule_btn = gr.Button("📅 產生今日行程")

    # 第六排
    schedule_output = gr.Textbox(lines=12, label="今日行程安排")

    # 第七排
    delete_task_dropdown = gr.Dropdown([], value=None, label="選擇要刪除的行程")

    # 第八排
    delete_task_btn = gr.Button("🗑️ 刪除指定行程")

    # 第九排
    clear_btn = gr.Button("🗑️ 清除所有行程")

    # 事件綁定
    add_btn.click(
        add_task,
        inputs=[category_input, subcategory_input, other_input, time_input],
        outputs=[task_table, category_input, subcategory_input, other_input, time_input, delete_task_dropdown]
    )

    schedule_btn.click(generate_schedule, start_time, schedule_output)

    delete_task_btn.click(
        delete_task,
        inputs=[delete_task_dropdown, start_time],
        outputs=[task_table, delete_task_dropdown, schedule_output]
    )

    clear_btn.click(
        clear_all,
        outputs=[
            task_table,
            category_input,
            subcategory_input,
            other_input,
            time_input,
            delete_task_dropdown,
            start_time,
            schedule_output
        ]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4c3471faa88a2f3022.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
